# Import packages

In [5]:
import pandas as pd
import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.slurm_manager import slurm_manager
from aqc_utils.helper_classes import slurm_status
from dft_descriptors import prepocessing as pp
import logging
logging.basicConfig(level=logging.INFO)

# Initialize the slurm manager

Slurm manager is going to manage what jobs you have currently running, or created. It caches your information, so you can kill the notebook, turn-off your computer, go on vacation, and pick it up later. It will remember which jobs you haven't retrieved from the cluster, etc.

Note: it won't ask you for login unless you execute a method that requires remote access

In [6]:
sm=slurm_manager(user='jschleinitz', host='hermite.chimie.ens.fr')
sm.connect()

INFO:aqc_utils.slurm_manager:Creating connection to hermite.chimie.ens.fr as jschleinitz
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:aqc_utils.slurm_manager:Connected to hermite.chimie.ens.fr as jschleinitz is True.


# Provide smiles string(s)
and copy into them into my_smiles variable. It's a list of strings (a Python list can be of any size), the smiles are extracted for the csv resulting form the step 2 numbering the molecules.

In [7]:
react_nb = pd.read_csv("../data_csv/Data_test10222021.csv", sep=',')
import numpy as np
from rdkit import Chem
df = pd.read_csv('../data_csv/Data_test10222021.csv', sep = ',')
df = df[df['Ligand effectif'].notna()]

In [8]:
unik_lig = [pp.dict_ligand[i] for i in np.unique(df["Ligand effectif"])]
can_unik_lig = np.unique([Chem.CanonSmiles(smi) for smi in unik_lig])
my_smiles = can_unik_lig
can_unik_lig
# determine if ligand is NHC, Phosphine, Diphosphine or other
def lig_type(list_lig):
    lig_type = []
    for smi in list_lig:
        m = Chem.MolFromSmiles(smi)
        if m.HasSubstructMatch(Chem.MolFromSmiles('N[C]N')):
            typ = 'NHC'
            if m.HasSubstructMatch(Chem.MolFromSmiles('P')):
                typ = 'mixed'
        elif m.HasSubstructMatch(Chem.MolFromSmiles('P')):
            typ = 'Phos'
            if len(m.GetSubstructMatches(Chem.MolFromSmiles('P'))) == 2:
                typ = 'DiPhos'
                if len(m.GetSubstructMatches(Chem.MolFromSmiles('P'))) > 2:
                    typ = 'mixed Phos'
        else:
            typ = 'other'
        lig_type.append(typ)
    return lig_type
phos = [can_unik_lig[i] for i in range(len(can_unik_lig)) if lig_type(can_unik_lig)[i] == 'Phos']
phos

['C1=CC(P(C2C=CC=C2)C2C=CC=C2)C=C1',
 'C1CCC(P(C2CCCCC2)C2CCCCC2)CC1',
 'C1CCC(P(C2CCCCC2)C2CCCCC2)CC1.CC(C)(C)N1C=CN(C(C)(C)C)C1',
 'C1CCC(P(C2CCCCC2)C2CCCCC2)CC1.CC(C)c1cccc(C(C)C)c1N1C=CN(c2c(C(C)C)cccc2C(C)C)C1',
 'C1CCC(P(C2CCCCC2)C2CCCCC2)CC1.[CH]1[CH][CH][C](P(c2ccccc2)c2ccccc2)[CH]1.[CH]1[CH][CH][C](P(c2ccccc2)c2ccccc2)[CH]1.[Fe]',
 'CC(C)(C)N1C=CN(C(C)(C)C)C1.c1ccc(P(c2ccccc2)c2ccccc2)cc1',
 'CC(C)(C)P(C(C)(C)C)C(C)(C)C',
 'CC(C)(C)P(C1CCCCC1)C1CCCCC1',
 'CC(C)(C)P(c1ccccc1-c1ccccc1)C(C)(C)C',
 'CC(C)CP(CC(C)C)CC(C)C',
 'CC(C)Oc1cccc(OC(C)C)c1-c1ccccc1P(C1CCCCC1)C1CCCCC1',
 'CC(C)P(C(C)C)C(C)C',
 'CC(C)P(C(C)C)C(Nc1ccccc1-n1nc(-c2ccccc2)cc1-c1ccccc1)c1ccccc1',
 'CC(C)c1cc(C(C)C)c(-c2cc(-c3c(C(C)C)cc(C(C)C)cc3C(C)C)cc(P(C3CCCC3)C3CCCC3)c2)c(C(C)C)c1',
 'CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)C2CCCCC2)c(C(C)C)c1',
 'CC(C)c1cccc(C(C)C)c1N1C=CN(c2c(C(C)C)cccc2C(C)C)C1.c1ccc(P(c2ccccc2)c2ccccc2)cc1',
 'CC(O)c1ccccc1P(c1ccccc1)c1ccccc1',
 'CCCCP(CCCC)CCCC',
 'CCP(CC)CC',
 'CN(C)P(N(C

# Initialize the molecules and generate conformations
We will use some for loops to loop over the smiles, only one conformation will be taken for each molecules in this work

In [10]:
mols = []
for smile in phos:
    mols.append(molecule(smile, max_num_conformers=1))

INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: C1=CC(C=C1)P(C1C=CC=C1)C1C=CC=C1
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: C1CCC(CC1)P(C1CCCCC1)C1CCCCC1
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: C1CCC(CC1)P(C1CCCCC1)C1CCCCC1.CC(N1C=CN(C1)C(C)(C)C)(C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'best'.
INFO:aqc_utils.molecule:Initial geometry created successfully.
INFO:aqc_utils.molecule:Molecule has 2 non-bonded fragments
INFO:aqc_utils.molecule:Initializing molecule with canonical smiles: C1CCC(CC1)P(C1CCCCC1)C1CCCCC1.CC(c1cccc(c1N1C=CN(C1)c1c(cccc1C(C)C)C(C)C)C(C)C)C
INFO:aqc_utils.molecule:Creating initial geometry with option 'b

Exception: Molecule has 4 non-bonded fragments. Only up to 2 are supported

###### Draw some molecules

In [ ]:
#mols[0].draw(2)

# Create gaussian job for each conformation

In [ ]:
for mol in mols:
    sm.create_jobs_for_molecule(mol, workflow_type="equilibrium")

In [ ]:
sm.get_job_stats(split_by_can=True)

In [ ]:
sm.submit_jobs()

In [ ]:
sm.squeue()

In [ ]:
for job in sm.get_jobs():
    if str(sm.get_jobs()[job].status) == 'slurm_status.done':
        print(sm.get_jobs()[job].base_name)

In [84]:
job

'6625625ec8945f514e3c68c6d47152bb'

In [85]:
job_all = sm.get_jobs()[job]

In [33]:
sm.get_job_stats()

status,done,failed,incomplete,uploaded
jobs,1,338,1,19


In [87]:
str(job_all.status)

'slurm_status.created'

In [88]:
job_all.base_name

'C12H13NO2_b9f2_conf_0'